In [27]:
import pandas as pd
import json

In [29]:
path = r"C:\Users\l1f22bscs0387\Desktop\movie_hit_predictor\data_cleaned\first_cleaned_dataset.csv"
clean_df1 = pd.read_csv(path)
clean_df1.columns

Index(['Unnamed: 0', 'budget', 'popularity', 'vote_count',
       'top_cast_avg_rating', 'director_encoded', 'release_year',
       'release_month', 'Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie',
       'Thriller', 'War', 'Western', 'title', 'genres', 'director', 'top_cast',
       'release_date', 'vote_average', 'revenue', 'budget_log', 'revenue_log',
       'profit_ratio'],
      dtype='object')

In [30]:
path_2 = r"C:\Users\l1f22bscs0387\Desktop\movie_hit_predictor\data_cleaned\second_cleaned_dataset.csv"
clean_df2 = pd.read_csv(path_2)
clean_df2.columns

Index(['Unnamed: 0', 'title', 'budget_log', 'revenue_log', 'profit_ratio',
       'popularity', 'vote_average', 'vote_count', 'director_encoded',
       'top_cast_avg_rating', 'release_year', 'release_month', 'Action',
       'Action.1', 'Action.2', 'Action.3', 'Adventure', 'Adventure.1',
       'Adventure.2', 'Adventure.3', 'Animation', 'Animation.1', 'Animation.2',
       'Animation.3', 'Comedy', 'Comedy.1', 'Comedy.2', 'Comedy.3', 'Crime',
       'Crime.1', 'Crime.2', 'Crime.3', 'Documentary', 'Documentary.1',
       'Documentary.2', 'Documentary.3', 'Drama', 'Drama.1', 'Drama.2',
       'Drama.3', 'Family', 'Family.1', 'Family.2', 'Family.3', 'Fantasy',
       'Fantasy.1', 'Fantasy.2', 'Fantasy.3', 'Foreign', 'Foreign.1',
       'Foreign.2', 'Foreign.3', 'History', 'History.1', 'History.2',
       'History.3', 'Horror', 'Horror.1', 'Horror.2', 'Horror.3', 'Music',
       'Music.1', 'Music.2', 'Music.3', 'Mystery', 'Mystery.1', 'Mystery.2',
       'Mystery.3', 'Romance', 'Romance.1'

In [31]:
# Compare columns between dataset 1 and 2
set1 = set(clean_df1.columns)
set2 = set(clean_df2.columns)

common_cols = set1.intersection(set2)
only_in_df1 = set1 - set2
only_in_df2 = set2 - set1

print("✅ COMMON COLUMNS (in both datasets):")
print(sorted(common_cols))
print("\n➕ ONLY IN DATASET 1:")
print(sorted(only_in_df1))
print("\n➕ ONLY IN DATASET 2:")
print(sorted(only_in_df2))


✅ COMMON COLUMNS (in both datasets):
['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'Unnamed: 0', 'War', 'Western', 'budget_log', 'director_encoded', 'popularity', 'profit_ratio', 'release_month', 'release_year', 'revenue_log', 'title', 'top_cast_avg_rating', 'vote_average', 'vote_count']

➕ ONLY IN DATASET 1:
['budget', 'director', 'genres', 'release_date', 'revenue', 'top_cast']

➕ ONLY IN DATASET 2:
['Action.1', 'Action.2', 'Action.3', 'Adventure.1', 'Adventure.2', 'Adventure.3', 'Animation.1', 'Animation.2', 'Animation.3', 'Comedy.1', 'Comedy.2', 'Comedy.3', 'Crime.1', 'Crime.2', 'Crime.3', 'Documentary.1', 'Documentary.2', 'Documentary.3', 'Drama.1', 'Drama.2', 'Drama.3', 'Family.1', 'Family.2', 'Family.3', 'Fantasy.1', 'Fantasy.2', 'Fantasy.3', 'Foreign.1', 'Foreign.2', 'Foreign.3', 'History.1', 'History.2', 'History.3', '

In [33]:
import numpy as np
import pandas as pd

# ✅ List of all main genre names
genre_cols = [
    'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
    'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery',
    'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western'
]

# ✅ Main features
feature_cols = [
    'title', 'budget', 'revenue', 'popularity', 'vote_average', 'vote_count',
    'director_encoded', 'top_cast_avg_rating', 'release_year', 'release_month',
    'budget_log', 'revenue_log', 'profit_ratio'
]

# 🩵 Step 1: Fix genre columns in clean_df2
for g in genre_cols:
    dupes = [c for c in clean_df2.columns if c.startswith(g)]
    if dupes:
        clean_df2[g] = clean_df2[dupes].sum(axis=1).clip(0, 1)
        clean_df2.drop(columns=dupes, inplace=True, errors='ignore')
    else:
        clean_df2[g] = 0  # add if not found

# 🩵 Step 2: Add missing numeric columns in both
for df in [clean_df1, clean_df2]:
    for col in ['budget', 'revenue']:
        if col not in df.columns:
            df[col] = np.expm1(df[col + '_log']) if col + '_log' in df.columns else 0

    if 'budget_log' not in df.columns:
        df['budget_log'] = np.log1p(df['budget'])
    if 'revenue_log' not in df.columns:
        df['revenue_log'] = np.log1p(df['revenue'])
    if 'profit_ratio' not in df.columns:
        df['profit_ratio'] = (df['revenue'] - df['budget']) / df['budget'].replace(0, np.nan)

# 🩵 Step 3: Add any missing genre columns
for g in genre_cols:
    if g not in clean_df1.columns:
        clean_df1[g] = 0
    if g not in clean_df2.columns:
        clean_df2[g] = 0

# 🩵 Step 4: Align columns
keep_cols = feature_cols + genre_cols
clean_df1 = clean_df1.reindex(columns=keep_cols, fill_value=0)
clean_df2 = clean_df2.reindex(columns=keep_cols, fill_value=0)

# 🩵 Step 5: Combine datasets
combined_df = pd.concat([clean_df1, clean_df2], ignore_index=True)

print(f"✅ Combined dataset shape: {combined_df.shape}")
print(f"✅ Columns: {combined_df.columns.tolist()}")
combined_df.head()


✅ Combined dataset shape: (49525, 33)
✅ Columns: ['title', 'budget', 'revenue', 'popularity', 'vote_average', 'vote_count', 'director_encoded', 'top_cast_avg_rating', 'release_year', 'release_month', 'budget_log', 'revenue_log', 'profit_ratio', 'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']


,title,budget,revenue,popularity,vote_average,vote_count,director_encoded,top_cast_avg_rating,release_year,release_month,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,Pirates of the Caribbean: At World's End,300000000.0,961000000.0,139.082615,6.9,4500.0,714,6.271936,2007.0,5.0,...,0,0,0,0,0,0,0,0,0,0
1,John Carter,260000000.0,284139100.0,43.926995,6.1,2124.0,89,6.108333,2012.0,3.0,...,0,0,0,0,0,1,0,0,0,0
2,Spider-Man 3,258000000.0,890871626.0,115.699814,5.9,3576.0,1838,6.271936,2007.0,5.0,...,0,0,0,0,0,0,0,0,0,0
3,Tangled,260000000.0,591794936.0,48.681969,7.4,3330.0,257,6.636667,2010.0,11.0,...,0,0,0,0,0,0,0,0,0,0
4,Superman Returns,270000000.0,391081192.0,57.925623,5.4,1400.0,255,5.685556,2006.0,6.0,...,0,0,0,0,0,1,0,0,0,0


In [34]:
combined_df.columns

Index(['title', 'budget', 'revenue', 'popularity', 'vote_average',
       'vote_count', 'director_encoded', 'top_cast_avg_rating', 'release_year',
       'release_month', 'budget_log', 'revenue_log', 'profit_ratio', 'Action',
       'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery',
       'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western'],
      dtype='object')

In [35]:
combined_df.head(50)

,title,budget,revenue,popularity,vote_average,vote_count,director_encoded,top_cast_avg_rating,release_year,release_month,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,Pirates of the Caribbean: At World's End,300000000.0,9.610000e+08,139.082615,6.9,4500.0,714,6.271936,2007.0,5.0,...,0,0,0,0,0,0,0,0,0,0
1,John Carter,260000000.0,2.841391e+08,43.926995,6.1,2124.0,89,6.108333,2012.0,3.0,...,0,0,0,0,0,1,0,0,0,0
2,Spider-Man 3,258000000.0,8.908716e+08,115.699814,5.9,3576.0,1838,6.271936,2007.0,5.0,...,0,0,0,0,0,0,0,0,0,0
3,Tangled,260000000.0,5.917949e+08,48.681969,7.4,3330.0,257,6.636667,2010.0,11.0,...,0,0,0,0,0,0,0,0,0,0
4,Superman Returns,270000000.0,3.910812e+08,57.925623,5.4,1400.0,255,5.685556,2006.0,6.0,...,0,0,0,0,0,1,0,0,0,0
5,Quantum of Solace,200000000.0,5.860907e+08,107.928811,6.1,2965.0,1284,6.271936,2008.0,10.0,...,0,0,0,0,0,0,0,1,0,0
6,Pirates of the Caribbean: Dead Man's Chest,200000000.0,1.065660e+09,145.847379,7.0,5246.0,714,6.271936,2006.0,6.0,...,0,0,0,0,0,0,0,0,0,0
7,The Lone Ranger,255000000.0,8.928991e+07,49.046956,5.9,2311.0,714,6.271936,2013.0,7.0,...,0,0,0,0,0,0,0,0,0,1
8,Man of Steel,225000000.0,6.628455e+08,99.398009,6.5,6359.0,2175,6.238889,2013.0,6.0,...,0,0,0,0,0,1,0,0,0,0
9,The Chronicles of Narnia: Prince Caspian,225000000.0,4.196514e+08,53.978602,6.3,1630.0,74,6.300000,2008.0,5.0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# Save as CSV (recommended)
combined_df.to_csv("combined_movies_cleaned.csv", index=False)

print("✅ Saved as 'combined_movies_cleaned.csv'")


✅ Saved as 'combined_movies_cleaned.csv'
